In [2]:
import mariadb
import os
import pandas as pd
import yaml
from columnar import columnar
from csv import DictReader
from datetime import date, timedelta
from dotenv import load_dotenv
from pandas.tseries.holiday import USFederalHolidayCalendar

_ = load_dotenv("config.env")
with open("config.yaml", "r") as stream:
    config = yaml.safe_load(stream)
    
DATABASE_NAME: str = "data_mart"

In [3]:
connection = mariadb.connect(
    host="127.0.0.1",
    port=23306,
    user=os.getenv("user"),
    password=os.getenv("password"),
    autocommit=False,
)
cursor = connection.cursor()

## Deliverable 2a

### Insert product class file into MariaDB

In [ ]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS {db}.product_class (
        product_class_id INT UNSIGNED PRIMARY KEY NOT NULL,
        product_subcategory TEXT NOT NULL,
        product_category TEXT NOT NULL,
        product_department TEXT NOT NULL,
        product_family TEXT NOT NULL
    )
    """.format(
        db=DATABASE_NAME
    )
)

with open("product_class.txt") as csv_file:
    csv_reader = DictReader(csv_file, delimiter="\t")
    for row in csv_reader:
        result = cursor.execute(
            """
            INSERT IGNORE INTO {db}.product_class (product_class_id, product_subcategory, product_category, product_department, product_family) VALUES (?, ?, ?, ?, ?)
            """.format(
                db=DATABASE_NAME
            ),
            (
                row["product_class_id"],
                row["product_subcategory"].upper(),
                row["product_category"].upper(),
                row["product_department"].upper(),
                row["product_family"].upper(),
            ),
        )
connection.commit()

### Create Product Dimension table in MariaDB

In [ ]:
cursor.execute("DROP TABLE IF EXISTS {db}.product_dimension".format(db=DATABASE_NAME))
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS {db}.product_dimension (
        product_key INT PRIMARY KEY AUTO_INCREMENT,
        sku INT UNSIGNED NOT NULL,
        product_name TEXT NOT NULL,
        product_class_id INT UNSIGNED,
        product_subcategory TEXT,
        product_category TEXT,
        product_department TEXT,
        product_family TEXT,
        size TEXT NOT NULL,
        number_per_case INT UNSIGNED NOT NULL,
        brand_name TEXT,
        manufacturer TEXT NOT NULL,
        supplier TEXT NOT NULL,
        product_class_source_key INT UNSIGNED NOT NULL
    )
    """.format(
        db=DATABASE_NAME
    )
)

### Create Product Class Source table in MariaDB

In [ ]:
cursor.execute(
    "DROP TABLE IF EXISTS {db}.product_class_source".format(db=DATABASE_NAME)
)
cursor.execute(
    """
    CREATE TABLE {db}.product_class_source (
        source_key INT PRIMARY KEY NOT NULL,
        source TEXT NOT NULL
    )
    """.format(
        db=DATABASE_NAME
    )
)


query = (
    "INSERT INTO {db}.product_class_source (source_key, source) VALUES (?, ?)".format(
        db=DATABASE_NAME
    )
)

records_to_insert = [
    (0, "NO PRODUCT CLASS"),
    (1, "FROM PRODUCT TABLE"),
    (2, "SPECIFIC ITEM TYPE MAPPED TO SPECIFIC PRODUCT SUBCATEGORY"),
    (3, "SPECIFIC MANUFACTURER MAPPED TO SPECIFIC PRODUCT SUBCATEGORY"),
    (4, "BASED ON ITEM TYPE AND STRING SEARCH OF PRODUCT NAME"),
    (5, "BASED ON MANUFACTURER AND STRING SEARCH OF PRODUCT NAME"),
    (6, "BASED ON STRING SEARCH OF PRODUCT NAME"),
]

cursor.executemany(query, records_to_insert)
connection.commit()

### Populate Product Dimension table in MariaDB

In [ ]:
cursor.execute("SELECT * FROM {db}.product_catalog".format(db=DATABASE_NAME))

type_replacements = config.get("typeReplacements")
manufacturer_to_class = config.get("manufacturerToClass")
product_name_to_class = config.get("productNameToClass")

count = 0
for row in [
    {cursor.description[index][0]: column for index, column in enumerate(value)}
    for value in cursor.fetchall()
]:
    item_type = row["item_type"]
    source = 1

    if item_type in type_replacements:
        item_type = type_replacements[item_type]
        source = 2

    cursor.execute(
        "SELECT * FROM {db}.product_class WHERE product_subcategory='{item_type}'".format(
            db=DATABASE_NAME, item_type=item_type
        )
    )
    match = cursor.fetchall()

    if len(match) == 0:

        if item_type == "COFFEE/CREAMER" and "coffee" in row["product_name"].lower():
            item_type = "COFFEE"
            source = 4
        elif item_type == "FROZEN FOOD":
            if "waffle" in row["product_name"].lower():
                item_type = "WAFFLES"
                source = 4
        elif item_type == "JELLY/JAM":
            if "jam" in row["product_name"].lower():
                item_type = "JAM"
                source = 4
            elif "jelly" in row["product_name"].lower():
                item_type = "JELLY"
                source = 4
        elif item_type == "SNACKS":
            if "variety pack" in row["product_name"].lower():
                item_type = "CHIPS"
                source = 4
        elif item_type == "SODA/JUICE/DRINKS":
            if "cocoa" in row["product_name"].lower():
                item_type = "CHOCOLATE"
                source = 4

        # rules by manufacturer
        if row["manufacturer"] in manufacturer_to_class:
            item_type = manufacturer_to_class[row["manufacturer"]]
            source = 3
        elif (
            row["manufacturer"] == "Starbucks"
            and "cocoa" in row["product_name"].lower()
        ):
            item_type = "CHOCOLATE"
            source = 5

        # rules by product_name
        for keyword in product_name_to_class.keys():
            if keyword in row["product_name"].lower():
                item_type = product_name_to_class[keyword]
                source = 6

        cursor.execute(
            "SELECT * FROM {db}.product_class WHERE product_subcategory='{item_type}'".format(
                db=DATABASE_NAME, item_type=item_type
            )
        )
        match = cursor.fetchall()

    if len(match) == 0:
        source = 0

    cursor.execute(
        """
        INSERT INTO {db}.product_dimension (sku, product_name, product_class_id, product_subcategory, product_category, product_department,
        product_family, size, number_per_case, brand_name, manufacturer, supplier, product_class_source_key)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """.format(
            db=DATABASE_NAME
        ),
        (
            row["sku"],
            row["product_name"],
            0 if len(match) == 0 else match[0][0],
            None if len(match) == 0 else match[0][1],
            None if len(match) == 0 else match[0][2],
            None if len(match) == 0 else match[0][3],
            None if len(match) == 0 else match[0][4],
            row["size"],
            12,
            None,
            row["manufacturer"],
            "Rowan Dairy" if row["item_type"] == "MILK" else "Rowan Warehouse",
            source,
        ),
    )
    connection.commit()

## Deliverable 2b

In [ ]:
cursor.execute("SELECT * FROM {db}.product_class_source".format(db=DATABASE_NAME))
for row in cursor.fetchall():
    print(row)

## Deliverable 3

In [ ]:
cursor.execute("DROP TABLE IF EXISTS {db}.stores".format(db=DATABASE_NAME))

cursor.execute(
    """
    CREATE TABLE {db}.stores (
        store_key SMALLINT PRIMARY KEY NOT NULL,
        store_manager TINYTEXT NOT NULL,
        street_address TINYTEXT NOT NULL,
        city TINYTEXT NOT NULL,
        state TINYTEXT NOT NULL,
        zip_code TINYTEXT NOT NULL,
        phone_number TINYTEXT NOT NULL
    )
    """.format(
        db=DATABASE_NAME
    )
)

query = (
    "INSERT INTO {db}.stores (store_key, store_manager, street_address, city, state, zip_code, phone_number) VALUES (?, ?, ?, ?, ?, ?, ?)".format(
        db=DATABASE_NAME
    )
)

records_to_insert = [
    (7, "Eric Mayo", "placeholder", "placeholder", "placeholder", "placeholder", "placeholder"),
    (8, "Jason Snouffer", "1640 Riverside Drive", "Hill Valley", "CA", "91905", "831-555-4385"),
]

cursor.executemany(query, records_to_insert)
connection.commit()

## Deliverable 4

In [ ]:
cursor.execute("DROP TABLE IF EXISTS {db}.dates".format(db=DATABASE_NAME))
cursor.execute(
    """
    CREATE TABLE {db}.dates (
        date_key SMALLINT PRIMARY KEY NOT NULL,
        datetime DATE NOT NULL,
        day_in_month SMALLINT NOT NULL,
        day_in_year SMALLINT NOT NULL,
        week_number SMALLINT  NOT NULL,
        month_number SMALLINT NOT NULL,
        month_text TINYTEXT NOT NULL,
        quarter SMALLINT NOT NULL,
        year SMALLINT NOT NULL,
        fiscal_year SMALLINT NOT NULL,
        is_holiday BOOLEAN NOT NULL,
        is_weekend BOOLEAN NOT NULL,
        season TINYTEXT NOT NULL
    )
    """.format(
        db=DATABASE_NAME
    )
)

In [ ]:
year = 2020
current_date: date = date(year, 1, 1)
stop_date: date = date(year, 12, 31)
fiscal_year_start = date(year, 8, 1)

holidays = USFederalHolidayCalendar().holidays(start=current_date.isoformat(), end=stop_date.isoformat()).to_pydatetime()
holidays = [d.date() for d in holidays]

seasons = [('winter', (date(year,  1,  1),  date(year,  3, 20))),
           ('spring', (date(year,  3, 21),  date(year,  6, 20))),
           ('summer', (date(year,  6, 21),  date(year,  9, 22))),
           ('autumn', (date(year,  9, 23),  date(year, 12, 20))),
           ('winter', (date(year, 12, 21),  date(year, 12, 31)))]

while current_date <= stop_date:   
    time_tuple = current_date.timetuple()
    weekday = current_date.strftime("%A")
    
    cursor.execute(
        """
        INSERT IGNORE INTO {db}.dates (
            date_key, datetime, day_in_month, day_in_year, week_number, month_number, month_text,
            quarter, year, fiscal_year, is_holiday, is_weekend, season
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """.format(
            db=DATABASE_NAME
        ),
        (
            time_tuple.tm_yday,
            current_date.isoformat(),
            time_tuple.tm_mday,
            time_tuple.tm_yday,
            current_date.isocalendar()[1],
            time_tuple.tm_mon,
            current_date.strftime("%B"),
            pd.Timestamp(current_date).quarter,
            time_tuple.tm_year,
            time_tuple.tm_year if current_date >= fiscal_year_start else time_tuple.tm_year - 1,
            True if current_date in holidays else False,
            True if weekday == "Saturday" or weekday == "Sunday" else False,
            next(season for season, (start, end) in seasons if start <= current_date <= end)
        ),
    )

    current_date = current_date + timedelta(days=1)
connection.commit()

In [ ]:
cursor.execute("SELECT * FROM {db}.dates LIMIT 25".format(db=DATABASE_NAME))
for row in cursor.fetchall():
    print(row)

## Deliverable 5

In [ ]:
cursor.execute("DROP TABLE IF EXISTS {db}.sales_facts_transaction_level".format(db=DATABASE_NAME))
cursor.execute(
    """
    CREATE TABLE {db}.sales_facts_transaction_level (
        date_key SMALLINT UNSIGNED NOT NULL,
        customer_number BIGINT UNSIGNED NOT NULL,
        product_key INT UNSIGNED NOT NULL,
        store_key SMALLINT UNSIGNED NOT NULL,
        quantity_sold SMALLINT UNSIGNED NOT NULL,
        total_dollar_sales DECIMAL(65,2) UNSIGNED NOT NULL,
        total_cost_to_store DECIMAL(65,2) UNSIGNED NOT NULL,
        gross_profit DECIMAL(65,2) UNSIGNED NOT NULL,
        PRIMARY KEY (date_key, customer_number, product_key)
    )
    SELECT date_key, customer_id AS customer_number, product_key, store_key, quantity_sold, total_dollar_sales, total_cost_to_store, gross_profit FROM
    (SELECT * FROM 
    (SELECT t.*, product_dimension.product_key, quantity_sold * base_price AS total_cost_to_store, total_dollar_sales - (quantity_sold * base_price) as gross_profit FROM
        (SELECT transactions.*, COUNT(*) AS quantity_sold, SUM(sale_price) AS total_dollar_sales, base_price
        FROM team8.transactions
        JOIN {db}.product_catalog ON transactions.sku = product_catalog.sku
        WHERE transaction_date BETWEEN '2020-12-01' AND '2020-12-31'
        GROUP BY sku, customer_id) t
        JOIN {db}.product_dimension ON t.sku = product_dimension.sku) t
    JOIN {db}.dates ON t.transaction_date = dates.datetime) t
    JOIN {db}.stores ON store_key = 8
    """.format(
        db=DATABASE_NAME
    )
)
connection.commit()

In [ ]:
cursor.execute(
    "SELECT * FROM {db}.sales_facts_transaction_level LIMIT 25".format(db=DATABASE_NAME)
)
tuples = cursor.fetchall()
lists = [list(x) for x in tuples]
table = columnar(
    lists,
    [
        "date_key",
        "customer_number",
        "product_key",
        "store_key",
        "quantity_sold",
        "total_dollar_sales",
        "total_cost_to_store",
        "gross_profit",
    ],
    no_borders=True,
    terminal_width=150,
)
print(table)

## Deliverable 6

In [4]:
cursor.execute(
    "DROP TABLE IF EXISTS {db}.sales_facts_daily_level".format(db=DATABASE_NAME)
)
cursor.execute(
    """
    CREATE TABLE {db}.sales_facts_daily_level (
        date_key SMALLINT UNSIGNED NOT NULL,
        product_key INT UNSIGNED NOT NULL,
        store_key SMALLINT UNSIGNED NOT NULL,
        number_sold_today SMALLINT UNSIGNED NOT NULL,
        cost_of_items_sold DECIMAL(65,2) UNSIGNED NOT NULL,
        sales_total DECIMAL(65,2) UNSIGNED NOT NULL,
        gross_profit DECIMAL(65,2) UNSIGNED NOT NULL,
        PRIMARY KEY (date_key, product_key, store_key)
    )
    """.format(
        db=DATABASE_NAME
    )
)

In [5]:
cursor.execute(
    """
    INSERT INTO {db}.sales_facts_daily_level (date_key, product_key, store_key, number_sold_today, cost_of_items_sold, sales_total, gross_profit)
    SELECT date_key, product_key, store_key, number_sold_today, cost_of_items_sold, sales_total, gross_profit FROM
    (SELECT * FROM 
    (SELECT t.*, product_dimension.product_key, number_sold_today * base_price AS cost_of_items_sold, sales_total - (number_sold_today * base_price) as gross_profit FROM
        (SELECT transactions.*, COUNT(*) AS number_sold_today, SUM(sale_price) AS sales_total, base_price
        FROM team8.transactions
        JOIN {db}.product_catalog ON transactions.sku = product_catalog.sku
        GROUP BY sku, transaction_date) t
        JOIN {db}.product_dimension ON t.sku = product_dimension.sku) t
    JOIN {db}.dates ON t.transaction_date = dates.datetime) t
    JOIN {db}.stores ON store_key = 8
    """.format(
        db=DATABASE_NAME
    )
)
connection.commit()

In [6]:
cursor.execute(
    """
    INSERT INTO {db}.sales_facts_daily_level (date_key, product_key, store_key, number_sold_today, cost_of_items_sold, sales_total, gross_profit)
    SELECT date_key, product_key, store_key, number_sold_today, cost_of_items_sold, sales_total, gross_profit FROM
    (SELECT * FROM 
    (SELECT t.*, product_dimension.product_key, number_sold_today * base_price AS cost_of_items_sold, sales_total - (number_sold_today * base_price) as gross_profit FROM
        (SELECT purchases.*, COUNT(*) AS number_sold_today, SUM(`Sale Price`) AS sales_total, base_price
        FROM team7.purchases
        JOIN {db}.product_catalog ON purchases.sku = product_catalog.sku
        GROUP BY sku, Date) t
        JOIN {db}.product_dimension ON t.sku = product_dimension.sku) t
    JOIN {db}.dates ON t.Date = dates.datetime) t
    JOIN {db}.stores ON store_key = 7
    """.format(
        db=DATABASE_NAME
    )
)
connection.commit()